In [14]:
import pandas as pd
import torch
import re
import shutil
from transformers import AutoTokenizer, AutoModelForCausalLM

file1 = "Employee_Report.xlsx"

# Backup original file
shutil.copy(file1, "backup_" + file1)

# Load ALL sheets
all_sheets = pd.read_excel(file1, sheet_name=None)

print("Sheets Loaded ✅")
print("Available Sheets:", list(all_sheets.keys()))


Sheets Loaded ✅
Available Sheets: ['Jan', 'Feb', 'Mar', 'Jan_Feb_Mar']


In [15]:
for sheet_name, df in all_sheets.items():
    df.columns = df.columns.str.strip()
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype(str).str.strip()
    for col in df.select_dtypes(include=["int64"]).columns:
        df[col] = df[col].astype(float)
    all_sheets[sheet_name] = df

print("All Sheets Cleaned ✅")


All Sheets Cleaned ✅


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11720\1407435403.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  for col in df.select_dtypes(include=["object"]).columns:
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11720\1407435403.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guid

In [16]:
model_name = "Qwen/Qwen2.5-3B-Instruct"  # Example local model

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

print("SLM Loaded ✅")


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


SLM Loaded ✅


In [25]:
question = input("Enter your Excel operation: ")

info_keywords = ["what is", "define", "explain", "meaning"]

if any(word in question.lower() for word in info_keywords):
    mode = "info"
else:
    mode = "excel"

print("Mode:", mode)


Enter your Excel operation:  Delete employee bala from all sheets.


Mode: excel


In [26]:
if mode == "excel":
    q_lower = question.lower()
    if "merge" in q_lower or "combine" in q_lower:
        operation_type = "merge"
    elif "delete" in q_lower or "remove" in q_lower:
        operation_type = "delete"
    elif "add" in q_lower or "insert" in q_lower:
        operation_type = "add"
    elif "update" in q_lower or "increase" in q_lower or "change" in q_lower:
        operation_type = "update"
    else:
        operation_type = "general"

    print("Detected Operation Type:", operation_type)


Detected Operation Type: delete


In [27]:
unsafe_keywords = ["import", "__", "os", "sys", "subprocess", "read_excel"]


In [28]:
if mode == "excel" and operation_type != "merge":

    for sheet_name in all_sheets.keys():

        print(f"\nProcessing Sheet: {sheet_name}")

        df1 = all_sheets[sheet_name]

        prompt = f"""
You are a pandas Excel automation engine.

STRICT RULES:
- Generate EXACTLY ONE line of Python code.
- Do NOT generate explanations or text.
- Do NOT generate multiple statements.
- Do NOT use reset_index or conditional logic (except delete condition).
- Do NOT use import or redefine df1.
- Only modify df1 directly.
- For adding rows use: df1.loc[len(df1)] = [values in correct column order].
- For deleting rows use: df1.drop(index=df1[CONDITION].index, inplace=True).

Columns:
{df1.columns.tolist()}

User Request:
{question}

Answer (Python code only, no text):
"""

        # Encode prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Generate output
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.1,
            do_sample=False
        )

        generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        # Clean code
        code = re.sub(r"```python|```", "", response).strip()
        lines = code.split("\n")
        code_lines = [line for line in lines if line.strip().startswith("df1")]
        if not code_lines:
            print("SLM did not generate executable code. Skipping ❌")
            continue
        code = code_lines[0]

        print("Final Code to Execute:", code)

        # Safety checks
        if any(word in code for word in unsafe_keywords):
            print("Unsafe code blocked ❌")
            continue

        if code.count("df1.") > 1 and operation_type != "delete":
            print("Multiple operations blocked ❌")
            continue

        # Execute safely
        try:
            exec(code, {"df1": df1})
            all_sheets[sheet_name] = df1
            print("Sheet Updated ✅")
        except Exception as e:
            print("Execution Error:", e)



Processing Sheet: Jan
Final Code to Execute: df1.drop(index=df1[(df1['Employee_Name'] == 'bala')].index, inplace=True)
Sheet Updated ✅

Processing Sheet: Feb
Final Code to Execute: df1.drop(index=df1[(df1['Employee_Name'] == 'bala')].index, inplace=True)
Sheet Updated ✅

Processing Sheet: Mar
Final Code to Execute: df1.drop(index=df1[(df1['Employee_Name'] == 'bala')].index, inplace=True)
Sheet Updated ✅

Processing Sheet: Jan_Feb_Mar
Final Code to Execute: df1.drop(index=df1[(df1['Employee_Name'] == 'bala')].index, inplace=True)
Sheet Updated ✅


In [29]:
if mode == "excel" and operation_type == "merge":
    # Detect sheets mentioned in question
    sheet_names = [name for name in all_sheets.keys() if name.lower() in question.lower()]
    if len(sheet_names) >= 2:
        df_merged = pd.concat([all_sheets[s] for s in sheet_names], ignore_index=True)
        merged_sheet_name = "_".join(sheet_names)
        all_sheets[merged_sheet_name] = df_merged
        print(f"Merged {sheet_names} into '{merged_sheet_name}' ✅")


In [30]:
if mode == "excel":
    with pd.ExcelWriter(file1, engine="openpyxl") as writer:
        for sheet_name, df in all_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print("All Sheets Updated Successfully ✅")


All Sheets Updated Successfully ✅


In [31]:
if mode == "info":
    print("\nInfo Response:\n")
    print(response)
